In [1]:
import funPytorch as fun
import configurationsMulticlass
import configurations
from tabulate import tabulate

In [2]:
def old():
    maxAcc = 0
    maxConf = ""
    for pfd in [2,4,8,16,32,64,128,256,512]:
        for afl in [1,2,4]:
            for afd in [2,4,8,16,32,64,128,256,512,1024,2048]:
                currConf = "{}-{}-{}".format(pfd, afl, afd)
                currMaxAcc = fun.getMaxTest(configurationsMulticlass.configTmulti1[pfd][afl][afd])
                print("{}:  \t{:.3f}".format(currConf, currMaxAcc))
                if currMaxAcc > maxAcc:
                    maxAcc = currMaxAcc
                    maxConf = currConf
                    maxEpoch = fun.getMaxValidEpoch(configurationsMulticlass.configTmulti1[pfd][afl][afd])
    print("======")
    print("{}:  \t{:.3f}\tepoch: {}".format(maxConf, maxAcc, maxEpoch))

In [3]:
def confExtractor(conf, par=[]):
    confs = []
    if type(conf) is dict:
        for k in conf:
            newPar = par.copy()
            newPar.append(k)
            confs.extend(confExtractor(conf[k], newPar))
    else:
        confs.append((conf,par))
    
    return confs

def calculate(conf, lim=None, useSet="test"): #lim list of single or lists, if -1 not limit
    maxAcc = 0
    maxPar = ""
    maxEpoch = ""
    
    maxAccPrint = 0
    maxParPrint = ""
    maxEpochPrint = ""
    
    allConfs = confExtractor(conf)
    
    if not lim is None:
        for i in range(len(lim)):
            if type(lim[i]) is list:
                for j in range(len(lim[i])):
                    found = False
                    for _,currPar in allConfs:
                        if lim[i][j] == currPar[i]:
                            found = True
                            break
                    if not found:
                        print("Parameter not found [{}][{}] ({})".format(i,j, lim[i][j]))
            elif lim[i] != -1:
                found = False
                for _,currPar in allConfs:
                    if lim[i] == currPar[i]:
                        found = True
                        break
                if not found:
                    print("Parameter not found [{}] ({})".format(i, lim[i]))

    toPrint = []
    for currConf, currPar in allConfs:
        ok = True
        if not lim is None:
            for i in range(len(currPar)):
                if type(lim[i]) is list:
                    if not currPar[i] in lim[i]:
                        ok = False
                else:
                    if not currPar[i] == lim[i] and not lim[i] == -1:
                        ok = False

        if useSet == 'test':
            currMaxAcc = fun.getMaxTest(currConf)
        elif useSet == 'valid':
            currMaxAcc = fun.getMaxValid(currConf)
        if ok:
            toPrint.append(["{}".format(currPar), "{:.3f}".format(currMaxAcc)])
            #print("{}:   \t{:.3f}".format(currPar, currMaxAcc))
            if currMaxAcc > maxAccPrint:
                maxAccPrint = currMaxAcc
                maxParPrint = currPar
                maxEpochPrint = fun.getMaxValidEpoch(currConf)
        if currMaxAcc > maxAcc:
            maxAcc = currMaxAcc
            maxPar = currPar
            maxEpoch = fun.getMaxValidEpoch(currConf)
            
    print(tabulate(toPrint, headers=['Parameters', 'Max acc.']))
    print("====== printed:")
    print("{}:  \t{:.3f}\tepoch: {}".format(maxParPrint, maxAccPrint, maxEpochPrint))
    print("====== tot:")
    print("{}:  \t{:.3f}\tepoch: {}".format(maxPar, maxAcc, maxEpoch))

# CNN

In [5]:
calculate(configurationsMulticlass.configTmultiCNNp, useSet='valid')

W0414 00:34:50.160742 139727066027840 deprecation.py:323] From /home/trianam/miniconda3/lib/python3.7/site-packages/tensorflow_core/python/summary/summary_iterator.py:68: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Parameters      Max acc.
------------  ----------
[64, 64]          91.171
[64, 128]         91.053
[64, 256]         90.906
[64, 512]         90.724
[128, 64]         91.394
[128, 128]        91.317
[128, 256]        91.118
[128, 512]        91.112
[256, 64]         91.311
[256, 128]        91.317
[256, 256]        91.335
[256, 512]        90.924
[512, 64]         91.27
[512, 128]        91.294
[512, 256]        90.953
[512, 512]        90.971
====== printed:
[128, 64]:  	91.394	epoch: 20
====== tot:
[128, 64]:  	91.394	epoch: 20


In [6]:
calculate(configurationsMulticlass.configTMmultiCNNp, useSet='valid')

Parameters      Max acc.
------------  ----------
[64, 64]          86.649
[64, 128]         86.337
[64, 256]         86.384
[64, 512]         85.99
[128, 64]         86.673
[128, 128]        86.19
[128, 256]        86.449
[128, 512]        86.125
[256, 64]         86.449
[256, 128]        86.255
[256, 256]        86.608
[256, 512]        86.007
[512, 64]         86.437
[512, 128]        86.632
[512, 256]        86.537
[512, 512]        85.948
====== printed:
[128, 64]:  	86.673	epoch: 9
====== tot:
[128, 64]:  	86.673	epoch: 9


# P

In [4]:
calculate(configurations.configP, [0,-1,2,128], useSet='valid')

W1110 21:24:08.081494 140715842271040 deprecation.py:323] From /home/trianam/miniconda3/lib/python3.7/site-packages/tensorflow/python/summary/summary_iterator.py:68: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Parameters          Max acc.
----------------  ----------
[0, 2, 2, 128]        57.143
[0, 4, 2, 128]        60.204
[0, 8, 2, 128]        72.449
[0, 16, 2, 128]       61.224
[0, 32, 2, 128]       61.224
[0, 64, 2, 128]       61.224
[0, 128, 2, 128]      62.245
[0, 256, 2, 128]      62.245
[0, 512, 2, 128]      62.245
[0, 8, 2, 128]:  	72.449	epoch: 63


# PHS

In [5]:
calculate(configurations.configPHS, [0,-1,-1,-1,-1], useSet='valid')

Parameters                Max acc.
----------------------  ----------
[0, 4, 0, 128, 4]           47.959
[0, 4, 0, 128, 8]           44.898
[0, 4, 0, 128, 16]          60.204
[0, 4, 0, 128, 32]          68.367
[0, 4, 0, 128, 64]          30.612
[0, 4, 0, 128, 128]         61.224
[0, 4, 0, 128, 256]         58.163
[0, 4, 0, 128, 512]         30.612
[0, 4, 1, 128, 4]           54.082
[0, 4, 1, 128, 8]           52.041
[0, 4, 1, 128, 16]          40.816
[0, 4, 1, 128, 32]          55.102
[0, 4, 1, 128, 64]          37.755
[0, 4, 1, 128, 128]         39.796
[0, 4, 1, 128, 256]         55.102
[0, 4, 1, 128, 512]         55.102
[0, 4, 1, 256, 4]           60.204
[0, 4, 1, 256, 8]           61.224
[0, 4, 1, 256, 16]          45.918
[0, 4, 1, 256, 32]          35.714
[0, 4, 1, 256, 64]          48.98
[0, 4, 1, 256, 128]         53.061
[0, 4, 1, 256, 256]         40.816
[0, 4, 1, 256, 512]         30.612
[0, 4, 1, 512, 4]           48.98
[0, 4, 1, 512, 8]           55.102
[0, 4, 1, 512, 16]    

# PHS2

In [4]:
calculate(configurations.configPHS2, [0,1,-1,1,-1,0,-1,-1], useSet='valid')

W1112 01:28:55.807147 140637863065408 deprecation.py:323] From /home/trianam/miniconda3/lib/python3.7/site-packages/tensorflow/python/summary/summary_iterator.py:68: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Parameters                          Max acc.
--------------------------------  ----------
[0, 1, 4, 1, 4, 0, 128, 4]            58.163
[0, 1, 4, 1, 4, 0, 128, 8]            54.082
[0, 1, 4, 1, 4, 0, 128, 16]           59.184
[0, 1, 4, 1, 4, 0, 128, 32]           56.122
[0, 1, 4, 1, 4, 0, 128, 64]           56.122
[0, 1, 4, 1, 4, 0, 128, 128]          48.98
[0, 1, 4, 1, 4, 0, 128, 256]          30.612
[0, 1, 4, 1, 4, 0, 128, 512]          53.061
[0, 1, 4, 1, 8, 0, 128, 4]            59.184
[0, 1, 4, 1, 8, 0, 128, 8]            56.122
[0, 1, 4, 1, 8, 0, 128, 16]           62.245
[0, 1, 4, 1, 8, 0, 128, 32]           48.98
[0, 1, 4, 1, 8, 0, 128, 64]           50
[0, 1, 4, 1, 8, 0, 128, 128]          48.98
[0, 1, 4, 1, 8, 0, 128, 256]          47.959
[0, 1, 4, 1, 8, 0, 128, 512]          30.612
[0, 1, 4, 1, 16, 0, 128, 4]           59.184
[0, 1, 4, 1, 16, 0, 128, 8]           58.163
[0, 1, 4, 1, 16, 0, 128, 16]          57.143
[0, 1, 4, 1, 16, 0, 128, 32]          57.143
[0, 1, 4, 1, 16, 

# PHS2_do

In [5]:
calculate(configurations.configPHS2_do, [0,1,-1,1,-1,0,-1,-1], useSet='valid')

Parameters                          Max acc.
--------------------------------  ----------
[0, 1, 4, 1, 4, 0, 128, 4]            48.98
[0, 1, 4, 1, 4, 0, 128, 8]            40.816
[0, 1, 4, 1, 4, 0, 128, 16]           60.204
[0, 1, 4, 1, 4, 0, 128, 32]           59.184
[0, 1, 4, 1, 4, 0, 128, 64]           59.184
[0, 1, 4, 1, 4, 0, 128, 128]          48.98
[0, 1, 4, 1, 4, 0, 128, 256]          46.939
[0, 1, 4, 1, 4, 0, 128, 512]          50
[0, 1, 4, 1, 8, 0, 128, 4]            60.204
[0, 1, 4, 1, 8, 0, 128, 8]            34.694
[0, 1, 4, 1, 8, 0, 128, 16]           57.143
[0, 1, 4, 1, 8, 0, 128, 32]           51.02
[0, 1, 4, 1, 8, 0, 128, 64]           54.082
[0, 1, 4, 1, 8, 0, 128, 128]          65.306
[0, 1, 4, 1, 8, 0, 128, 256]          40.816
[0, 1, 4, 1, 8, 0, 128, 512]          30.612
[0, 1, 4, 1, 16, 0, 128, 4]           61.224
[0, 1, 4, 1, 16, 0, 128, 8]           47.959
[0, 1, 4, 1, 16, 0, 128, 16]          56.122
[0, 1, 4, 1, 16, 0, 128, 32]          62.245
[0, 1, 4, 1, 16, 

# PHS2_350_paper

In [7]:
calculate(configurations.configPHS2_350_paper, useSet='valid')

Parameters      Max acc.
------------  ----------
[]                70.297
[]:  	70.297	epoch: 51


In [8]:
calculate(configurations.configPHS2_350_paper, useSet='test')

Parameters      Max acc.
------------  ----------
[]                68.889
[]:  	68.889	epoch: 51


# PHS2_350_noDo

In [6]:
calculate(configurations.configPHS2_350_noDo, [0,1,-1,1,-1,0,-1,-1], useSet='valid')

Parameters                          Max acc.
--------------------------------  ----------
[0, 1, 4, 1, 4, 0, 128, 4]            60.204
[0, 1, 4, 1, 4, 0, 128, 8]            61.224
[0, 1, 4, 1, 4, 0, 128, 16]           63.265
[0, 1, 4, 1, 4, 0, 128, 32]           59.184
[0, 1, 4, 1, 4, 0, 128, 64]           53.061
[0, 1, 4, 1, 4, 0, 128, 128]          53.061
[0, 1, 4, 1, 4, 0, 128, 256]          60.204
[0, 1, 4, 1, 4, 0, 128, 512]          30.612
[0, 1, 4, 1, 8, 0, 128, 4]            57.143
[0, 1, 4, 1, 8, 0, 128, 8]            56.122
[0, 1, 4, 1, 8, 0, 128, 16]           64.286
[0, 1, 4, 1, 8, 0, 128, 32]           56.122
[0, 1, 4, 1, 8, 0, 128, 64]           57.143
[0, 1, 4, 1, 8, 0, 128, 128]          32.653
[0, 1, 4, 1, 8, 0, 128, 256]          55.102
[0, 1, 4, 1, 8, 0, 128, 512]          57.143
[0, 1, 4, 1, 16, 0, 128, 4]           60.204
[0, 1, 4, 1, 16, 0, 128, 8]           61.224
[0, 1, 4, 1, 16, 0, 128, 16]          57.143
[0, 1, 4, 1, 16, 0, 128, 32]          54.082
[0, 1, 4, 

# PHS2_350

In [22]:
calculate(configurations.configPHS2_350, [9,1,-1,1,-1,0,-1,-1], useSet='valid')

Parameters                          Max acc.
--------------------------------  ----------
[9, 1, 4, 1, 4, 0, 128, 4]            68.317
[9, 1, 4, 1, 4, 0, 128, 8]            63.366
[9, 1, 4, 1, 4, 0, 128, 16]           64.356
[9, 1, 4, 1, 4, 0, 128, 32]           57.426
[9, 1, 4, 1, 4, 0, 128, 64]           58.416
[9, 1, 4, 1, 4, 0, 128, 128]          55.446
[9, 1, 4, 1, 4, 0, 128, 256]          54.455
[9, 1, 4, 1, 4, 0, 128, 512]          57.426
[9, 1, 4, 1, 8, 0, 128, 4]            64.356
[9, 1, 4, 1, 8, 0, 128, 8]            62.376
[9, 1, 4, 1, 8, 0, 128, 16]           63.366
[9, 1, 4, 1, 8, 0, 128, 32]           49.505
[9, 1, 4, 1, 8, 0, 128, 64]           63.366
[9, 1, 4, 1, 8, 0, 128, 128]          59.406
[9, 1, 4, 1, 8, 0, 128, 256]          53.465
[9, 1, 4, 1, 8, 0, 128, 512]          29.703
[9, 1, 4, 1, 16, 0, 128, 4]           62.376
[9, 1, 4, 1, 16, 0, 128, 8]           60.396
[9, 1, 4, 1, 16, 0, 128, 16]          66.337
[9, 1, 4, 1, 16, 0, 128, 32]          64.356
[9, 1, 4, 

In [23]:
calculate(configurations.configPHS2_350, [9,1,-1,1,-1,0,-1,-1], useSet='test')

Parameters                          Max acc.
--------------------------------  ----------
[9, 1, 4, 1, 4, 0, 128, 4]            65.556
[9, 1, 4, 1, 4, 0, 128, 8]            66.667
[9, 1, 4, 1, 4, 0, 128, 16]           61.111
[9, 1, 4, 1, 4, 0, 128, 32]           54.444
[9, 1, 4, 1, 4, 0, 128, 64]           57.778
[9, 1, 4, 1, 4, 0, 128, 128]          55.556
[9, 1, 4, 1, 4, 0, 128, 256]          52.222
[9, 1, 4, 1, 4, 0, 128, 512]          61.111
[9, 1, 4, 1, 8, 0, 128, 4]            57.778
[9, 1, 4, 1, 8, 0, 128, 8]            58.889
[9, 1, 4, 1, 8, 0, 128, 16]           63.333
[9, 1, 4, 1, 8, 0, 128, 32]           52.222
[9, 1, 4, 1, 8, 0, 128, 64]           60
[9, 1, 4, 1, 8, 0, 128, 128]          62.222
[9, 1, 4, 1, 8, 0, 128, 256]          54.444
[9, 1, 4, 1, 8, 0, 128, 512]          32.222
[9, 1, 4, 1, 16, 0, 128, 4]           56.667
[9, 1, 4, 1, 16, 0, 128, 8]           58.889
[9, 1, 4, 1, 16, 0, 128, 16]          61.111
[9, 1, 4, 1, 16, 0, 128, 32]          62.222
[9, 1, 4, 1, 1

# F

In [15]:
calculate(configurations.configFmulti1, [[64,128,256],[1,2],[128,256,512]], useSet='test')

Parameters       Max acc.
-------------  ----------
[64, 1, 128]       84.954
[64, 1, 256]       84.65
[64, 1, 512]       84.043
[64, 2, 128]       84.802
[64, 2, 256]       83.435
[64, 2, 512]       84.347
[128, 1, 128]      84.347
[128, 1, 256]      85.562
[128, 1, 512]      84.195
[128, 2, 128]      82.979
[128, 2, 256]      85.41
[128, 2, 512]      85.258
[256, 1, 128]      83.587
[256, 1, 256]      85.41
[256, 1, 512]      84.195
[256, 2, 128]      85.562
[256, 2, 256]      84.802
[256, 2, 512]      84.498
[128, 1, 256]:  	85.562	epoch: 2


# FHS

In [17]:
#calculate(configurations.configFHS, [[64,128,256],[1,2],[512,1024,2048],[64,128,256]])
calculate(configurations.configFHS, [[32,64,128],[1,2,4],[512,1024,2048],[128,256,512]], useSet='test')

Parameters             Max acc.
-------------------  ----------
[32, 1, 512, 128]        81.155
[32, 1, 512, 256]        81.459
[32, 1, 512, 512]        50
[32, 1, 1024, 128]       49.848
[32, 1, 1024, 256]       60.03
[32, 1, 1024, 512]       81.763
[32, 1, 2048, 128]       50
[32, 1, 2048, 256]       50
[32, 1, 2048, 512]       50
[32, 2, 512, 128]        50.152
[32, 2, 512, 256]        81.611
[32, 2, 512, 512]        50
[32, 2, 1024, 128]       80.851
[32, 2, 1024, 256]       82.675
[32, 2, 1024, 512]       83.131
[32, 2, 2048, 128]       50.152
[32, 2, 2048, 256]       50
[32, 2, 2048, 512]       50
[32, 4, 512, 128]        51.216
[32, 4, 512, 256]        50.152
[32, 4, 512, 512]        50
[32, 4, 1024, 128]       50
[32, 4, 1024, 256]       50
[32, 4, 1024, 512]       50
[32, 4, 2048, 128]       50
[32, 4, 2048, 256]       50.608
[32, 4, 2048, 512]       50
[64, 1, 512, 128]        80.395
[64, 1, 512, 256]        50
[64, 1, 512, 512]        81.307
[64, 1, 1024, 128]       50
[64, 

# B

In [23]:
calculate(configurations.configB,[[8,16,32],[2,4,8],[256,512,1024]], useSet='valid')

Parameter not found [1][2] (8)
Parameters       Max acc.
-------------  ----------
[8, 2, 256]        99.784
[8, 2, 512]        99.641
[8, 2, 1024]       -1
[8, 4, 256]        99.641
[8, 4, 512]        99.713
[8, 4, 1024]       -1
[16, 2, 256]       99.713
[16, 2, 512]       99.641
[16, 2, 1024]      -1
[16, 4, 256]       99.856
[16, 4, 512]       99.928
[16, 4, 1024]      -1
[32, 2, 256]       99.713
[32, 2, 512]       99.641
[32, 2, 1024]      -1
[32, 4, 256]       99.856
[32, 4, 512]       99.784
[32, 4, 1024]      -1
[16, 4, 512]:  	99.928	epoch: 9


In [24]:
calculate(configurations.configB,[[8,16,32],[1,2,4],[128,256,512]], useSet='test')

Parameters      Max acc.
------------  ----------
[8, 1, 128]       99.282
[8, 1, 256]       98.707
[8, 1, 512]       99.425
[8, 2, 128]       99.282
[8, 2, 256]       99.21
[8, 2, 512]       99.138
[8, 4, 128]       99.138
[8, 4, 256]       98.348
[8, 4, 512]       99.066
[16, 1, 128]      99.353
[16, 1, 256]      99.425
[16, 1, 512]      99.282
[16, 2, 128]      99.21
[16, 2, 256]      99.569
[16, 2, 512]      99.21
[16, 4, 128]      99.425
[16, 4, 256]      99.21
[16, 4, 512]      99.282
[32, 1, 128]      99.497
[32, 1, 256]      99.497
[32, 1, 512]      99.425
[32, 2, 128]      99.21
[32, 2, 256]      99.353
[32, 2, 512]      99.425
[32, 4, 128]      99.282
[32, 4, 256]      99.282
[32, 4, 512]      99.282
[16, 2, 256]:  	99.569	epoch: 2


# BHS

In [25]:
calculate(configurations.configBHS,[[8,16,32],[1,2],[256,512,1024],[32,64,128]], useSet='valid')

Parameters            Max acc.
------------------  ----------
[8, 1, 256, 32]         99.641
[8, 1, 256, 64]         99.784
[8, 1, 256, 128]        81.968
[8, 1, 512, 32]         81.968
[8, 1, 512, 64]         81.968
[8, 1, 512, 128]        99.713
[8, 1, 1024, 32]        81.968
[8, 1, 1024, 64]        81.968
[8, 1, 1024, 128]       73.132
[8, 2, 256, 32]         99.425
[8, 2, 256, 64]         81.968
[8, 2, 256, 128]        81.968
[8, 2, 512, 32]         81.968
[8, 2, 512, 64]         99.784
[8, 2, 512, 128]        99.784
[8, 2, 1024, 32]        99.713
[8, 2, 1024, 64]        73.132
[8, 2, 1024, 128]       99.641
[16, 1, 256, 32]        99.928
[16, 1, 256, 64]        73.132
[16, 1, 256, 128]       99.784
[16, 1, 512, 32]        99.713
[16, 1, 512, 64]        99.784
[16, 1, 512, 128]       73.132
[16, 1, 1024, 32]       99.713
[16, 1, 1024, 64]       81.968
[16, 1, 1024, 128]      99.784
[16, 2, 256, 32]        81.968
[16, 2, 256, 64]        81.968
[16, 2, 256, 128]       81.968
[16, 2, 

In [21]:
calculate(configurations.configBHS,[[8,16,32],[1,2],[256,512,1024],[32,64,128]], useSet='test')

Parameters            Max acc.
------------------  ----------
[8, 1, 256, 32]         98.348
[8, 1, 256, 64]         98.707
[8, 1, 256, 128]        84.626
[8, 1, 512, 32]         84.626
[8, 1, 512, 64]         84.626
[8, 1, 512, 128]        98.994
[8, 1, 1024, 32]        84.626
[8, 1, 1024, 64]        84.626
[8, 1, 1024, 128]       78.951
[8, 2, 256, 32]         98.348
[8, 2, 256, 64]         84.626
[8, 2, 256, 128]        84.626
[8, 2, 512, 32]         84.626
[8, 2, 512, 64]         98.922
[8, 2, 512, 128]        99.353
[8, 2, 1024, 32]        98.635
[8, 2, 1024, 64]        78.951
[8, 2, 1024, 128]       99.21
[16, 1, 256, 32]        99.138
[16, 1, 256, 64]        78.951
[16, 1, 256, 128]       99.21
[16, 1, 512, 32]        99.353
[16, 1, 512, 64]        99.497
[16, 1, 512, 128]       78.951
[16, 1, 1024, 32]       99.21
[16, 1, 1024, 64]       84.626
[16, 1, 1024, 128]      99.138
[16, 2, 256, 32]        84.626
[16, 2, 256, 64]        84.626
[16, 2, 256, 128]       84.626
[16, 2, 512

# B1000

In [26]:
calculate(configurations.configB1000,[[4,8,16],[2,4,8],[64,128,256]], useSet='test')

Parameters      Max acc.
------------  ----------
[4, 2, 64]            95
[4, 2, 128]           94
[4, 2, 256]           95
[4, 4, 64]            96
[4, 4, 128]           95
[4, 4, 256]           92
[4, 8, 64]            93
[4, 8, 128]           86
[4, 8, 256]           91
[8, 2, 64]            97
[8, 2, 128]           96
[8, 2, 256]           95
[8, 4, 64]            94
[8, 4, 128]           99
[8, 4, 256]           90
[8, 8, 64]            97
[8, 8, 128]           95
[8, 8, 256]           93
[16, 2, 64]           92
[16, 2, 128]          94
[16, 2, 256]          96
[16, 4, 64]           95
[16, 4, 128]          96
[16, 4, 256]          94
[16, 8, 64]           95
[16, 8, 128]          95
[16, 8, 256]          95
[8, 4, 128]:  	99.000	epoch: 91


# BHS1000

In [27]:
calculate(configurations.configBHS1000,[[8,16,32],[1,2],[256,512,1024],[8,16,32]], useSet='test')

Parameters           Max acc.
-----------------  ----------
[8, 1, 256, 8]             90
[8, 1, 256, 16]            79
[8, 1, 256, 32]            95
[8, 1, 512, 8]             95
[8, 1, 512, 16]            79
[8, 1, 512, 32]            84
[8, 1, 1024, 8]            92
[8, 1, 1024, 16]           81
[8, 1, 1024, 32]           92
[8, 2, 256, 8]             95
[8, 2, 256, 16]            93
[8, 2, 256, 32]            79
[8, 2, 512, 8]             93
[8, 2, 512, 16]            85
[8, 2, 512, 32]            81
[8, 2, 1024, 8]            95
[8, 2, 1024, 16]           84
[8, 2, 1024, 32]           81
[16, 1, 256, 8]            96
[16, 1, 256, 16]           81
[16, 1, 256, 32]           81
[16, 1, 512, 8]            96
[16, 1, 512, 16]           98
[16, 1, 512, 32]           85
[16, 1, 1024, 8]           92
[16, 1, 1024, 16]          96
[16, 1, 1024, 32]          84
[16, 2, 256, 8]            95
[16, 2, 256, 16]           81
[16, 2, 256, 32]           84
[16, 2, 512, 8]            97
[16, 2, 51

# B1000b

In [13]:
calculate(configurations.configB1000b,[[2,4,8],[1,2],[32,64,128]], useSet='valid')

Parameters      Max acc.
------------  ----------
[2, 1, 32]            99
[2, 1, 64]            99
[2, 1, 128]           98
[2, 2, 32]           100
[2, 2, 64]            98
[2, 2, 128]           95
[4, 1, 32]            98
[4, 1, 64]           100
[4, 1, 128]           96
[4, 2, 32]            96
[4, 2, 64]            96
[4, 2, 128]           99
[8, 1, 32]            99
[8, 1, 64]            97
[8, 1, 128]           96
[8, 2, 32]            94
[8, 2, 64]            98
[8, 2, 128]           99
[2, 1, 16]:  	100.000	epoch: 15


In [15]:
calculate(configurations.configB1000b,[4,1,64], useSet='test')

Parameters      Max acc.
------------  ----------
[4, 1, 64]            95
[128, 8, 128]:  	99.000	epoch: 29


# BHS1000b

In [16]:
calculate(configurations.configBHS1000b,[[4,8,16],[1,2,4],[128,256,512],[4,8,16]], useSet='valid')

Parameters          Max acc.
----------------  ----------
[4, 1, 128, 4]            97
[4, 1, 128, 8]            96
[4, 1, 128, 16]           99
[4, 1, 256, 4]            97
[4, 1, 256, 8]            95
[4, 1, 256, 16]           99
[4, 1, 512, 4]            95
[4, 1, 512, 8]            96
[4, 1, 512, 16]           95
[4, 2, 128, 4]            98
[4, 2, 128, 8]            96
[4, 2, 128, 16]           98
[4, 2, 256, 4]            96
[4, 2, 256, 8]            97
[4, 2, 256, 16]           99
[4, 2, 512, 4]            96
[4, 2, 512, 8]            98
[4, 2, 512, 16]           73
[4, 4, 128, 4]            93
[4, 4, 128, 8]            99
[4, 4, 128, 16]           94
[4, 4, 256, 4]            98
[4, 4, 256, 8]            99
[4, 4, 256, 16]           98
[4, 4, 512, 4]            95
[4, 4, 512, 8]            97
[4, 4, 512, 16]           98
[8, 1, 128, 4]            94
[8, 1, 128, 8]            96
[8, 1, 128, 16]           97
[8, 1, 256, 4]            98
[8, 1, 256, 8]            98
[8, 1, 256, 16

In [17]:
calculate(configurations.configBHS1000b,[4,1,256,32], useSet='test')

Parameters         Max acc.
---------------  ----------
[4, 1, 256, 32]          94
[16, 4, 256, 4]:  	99.000	epoch: 16


# H

In [9]:
calculate(configurationsMulticlass.configH)

[32, 0, 256]:   	88.829
[32, 1, 256]:   	89.375
[32, 1, 512]:   	89.228
[32, 1, 1024]:   	88.371
[32, 1, 2048]:   	88.682
[32, 2, 256]:   	89.152
[32, 2, 512]:   	88.946
[32, 2, 1024]:   	88.858
[32, 2, 2048]:   	88.248
[32, 4, 256]:   	87.884
[32, 4, 512]:   	85.594
[32, 4, 1024]:   	13.402
[32, 4, 2048]:   	20.047
[64, 0, 256]:   	89.422
[64, 1, 256]:   	89.345
[64, 1, 512]:   	89.257
[64, 1, 1024]:   	89.469
[64, 1, 2048]:   	89.263
[64, 2, 256]:   	89.598
[64, 2, 512]:   	89.316
[64, 2, 1024]:   	89.797
[64, 2, 2048]:   	88.236
[64, 4, 256]:   	88.805
[64, 4, 512]:   	87.449
[64, 4, 1024]:   	35.685
[64, 4, 2048]:   	28.242
[128, 0, 256]:   	89.369
[128, 1, 256]:   	89.704
[128, 1, 512]:   	89.645
[128, 1, 1024]:   	89.639
[128, 1, 2048]:   	89.381
[128, 2, 256]:   	89.451
[128, 2, 512]:   	89.486
[128, 2, 1024]:   	89.469
[128, 2, 2048]:   	88.588
[128, 4, 256]:   	88.107
[128, 4, 512]:   	13.361
[128, 4, 1024]:   	70.437
[128, 4, 2048]:   	13.038
[256, 0, 256]:   	88.935
[256, 1,

# HS

In [15]:
calculate(configurationsMulticlass.configHS)

[32, 0, 256, 64]:   	-1.000
[32, 0, 256, 128]:   	-1.000
[32, 0, 256, 256]:   	-1.000
[32, 0, 256, 512]:   	-1.000
[32, 1, 256, 64]:   	-1.000
[32, 1, 256, 128]:   	-1.000
[32, 1, 256, 256]:   	-1.000
[32, 1, 256, 512]:   	-1.000
[32, 1, 512, 64]:   	-1.000
[32, 1, 512, 128]:   	-1.000
[32, 1, 512, 256]:   	-1.000
[32, 1, 512, 512]:   	-1.000
[32, 1, 1024, 64]:   	-1.000
[32, 1, 1024, 128]:   	-1.000
[32, 1, 1024, 256]:   	-1.000
[32, 1, 1024, 512]:   	-1.000
[32, 1, 2048, 64]:   	-1.000
[32, 1, 2048, 128]:   	-1.000
[32, 1, 2048, 256]:   	-1.000
[32, 1, 2048, 512]:   	-1.000
[32, 2, 256, 64]:   	-1.000
[32, 2, 256, 128]:   	-1.000
[32, 2, 256, 256]:   	-1.000
[32, 2, 256, 512]:   	-1.000
[32, 2, 512, 64]:   	-1.000
[32, 2, 512, 128]:   	-1.000
[32, 2, 512, 256]:   	-1.000
[32, 2, 512, 512]:   	-1.000
[32, 2, 1024, 64]:   	18.192
[32, 2, 1024, 128]:   	12.633
[32, 2, 1024, 256]:   	12.838
[32, 2, 1024, 512]:   	-1.000
[32, 2, 2048, 64]:   	89.322
[32, 2, 2048, 128]:   	89.240
[32, 2, 2

# HM

In [5]:
calculate(configurationsMulticlass.configHM,[64,1,-1])

W1031 10:54:07.060907 140041015969600 deprecation.py:323] From /home/trianam/miniconda3/lib/python3.7/site-packages/tensorflow/python/summary/summary_iterator.py:68: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


[64, 1, 256]:   	82.739
[64, 1, 512]:   	82.638
[64, 1, 1024]:   	83.130
[64, 1, 2048]:   	81.814
[64, 1, 1024]:  	83.130	epoch: 12


# HSM

In [4]:
calculate(configurationsMulticlass.configHSM,[64,1,-1,128])

W1029 11:27:52.370705 140587285694272 deprecation.py:323] From /home/trianam/miniconda3/lib/python3.7/site-packages/tensorflow/python/summary/summary_iterator.py:68: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


[64, 1, 256, 128]:   	82.946
[64, 1, 512, 128]:   	82.869
[64, 1, 1024, 128]:   	83.177
[64, 1, 2048, 128]:   	-1.000
[64, 1, 1024, 128]:  	83.177	epoch: 17




# Tmulti1

In [74]:
calculate(configurationsMulticlass.configTmulti1)

[2, 1, 2]:   	67.948
[2, 1, 4]:   	75.280
[2, 1, 8]:   	77.581
[2, 1, 16]:   	80.904
[2, 1, 32]:   	83.927
[2, 1, 64]:   	84.632
[2, 1, 128]:   	86.375
[2, 1, 256]:   	84.884
[2, 1, 512]:   	86.305
[2, 1, 1024]:   	86.628
[2, 1, 2048]:   	86.252
[2, 2, 2]:   	72.639
[2, 2, 4]:   	68.312
[2, 2, 8]:   	80.305
[2, 2, 16]:   	83.258
[2, 2, 32]:   	85.207
[2, 2, 64]:   	85.424
[2, 2, 128]:   	86.827
[2, 2, 256]:   	86.305
[2, 2, 512]:   	85.700
[2, 2, 1024]:   	85.354
[2, 2, 2048]:   	85.277
[2, 4, 2]:   	45.401
[2, 4, 4]:   	56.302
[2, 4, 8]:   	75.421
[2, 4, 16]:   	82.665
[2, 4, 32]:   	85.436
[2, 4, 64]:   	85.835
[2, 4, 128]:   	86.815
[2, 4, 256]:   	81.579
[2, 4, 512]:   	84.790
[2, 4, 1024]:   	79.988
[2, 4, 2048]:   	78.620
[4, 1, 2]:   	73.748
[4, 1, 4]:   	80.781
[4, 1, 8]:   	85.365
[4, 1, 16]:   	86.293
[4, 1, 32]:   	87.185
[4, 1, 64]:   	88.406
[4, 1, 128]:   	88.483
[4, 1, 256]:   	88.224
[4, 1, 512]:   	88.324
[4, 1, 1024]:   	88.353
[4, 1, 2048]:   	88.459
[4, 2, 2]:   	75

# Tmulti2

In [5]:
calculate(configurationsMulticlass.configTmulti2)

-64-1-64:  	89.011
-64-1-128:  	88.970
-64-1-256:  	89.463
-64-1-512:  	89.175
-64-1-1024:  	89.733
-64-2-64:  	88.565
-64-2-128:  	88.641
-64-2-256:  	89.158
-64-2-512:  	89.486
-64-2-1024:  	89.187
-128-1-64:  	89.093
-128-1-128:  	89.498
-128-1-256:  	89.416
-128-1-512:  	89.621
-128-1-1024:  	89.827
-128-2-64:  	88.976
-128-2-128:  	88.700
-128-2-256:  	89.387
-128-2-512:  	89.680
-128-2-1024:  	89.398
-256-1-64:  	89.064
-256-1-128:  	89.657
-256-1-256:  	89.839
-256-1-512:  	89.639
-256-1-1024:  	89.569
-256-2-64:  	88.794
-256-2-128:  	89.545
-256-2-256:  	89.510
-256-2-512:  	89.345
-256-2-1024:  	88.970
-512-1-64:  	89.163
-512-1-128:  	89.850
-512-1-256:  	89.668
-512-1-512:  	89.463
-512-1-1024:  	89.610
-512-2-64:  	88.477
-512-2-128:  	89.075
-512-2-256:  	88.706
-512-2-512:  	89.287
-512-2-1024:  	89.357
-1024-1-64:  	89.070
-1024-1-128:  	88.823
-1024-1-256:  	89.387
-1024-1-512:  	89.058
-1024-1-1024:  	89.310
-1024-2-64:  	89.181
-1024-2-128:  	88.635
-1024-2-256:  	89

# TImulti1

In [6]:
calculate(configurationsMulticlass.configTImulti1)

-2-1-2:  	17.006
-2-2-2:  	25.090
-2-2-4:  	24.344
-2-2-8:  	9.357
-2-2-16:  	12.662
-2-2-32:  	24.943
-2-2-64:  	24.943
-2-2-128:  	41.192
-2-2-256:  	24.432
-2-2-512:  	24.555
-2-2-1024:  	24.943
-2-2-2048:  	14.752
-2-4-2:  	10.643
-2-4-4:  	24.943
-2-4-8:  	3.927
-2-4-16:  	3.933
-2-4-32:  	9.968
-2-4-64:  	12.633
-2-4-128:  	3.927
-2-4-256:  	3.933
-2-4-512:  	3.927
-2-4-1024:  	3.927
-2-4-2048:  	3.927
-4-1-2:  	25.401
-4-2-2:  	12.633
-4-2-4:  	12.633
-4-2-8:  	12.633
-4-2-16:  	12.633
-4-2-32:  	24.943
-4-2-64:  	9.357
-4-2-128:  	24.937
-4-2-256:  	24.943
-4-2-512:  	80.980
-4-2-1024:  	70.038
-4-2-2048:  	25.283
-4-4-2:  	24.943
-4-4-4:  	9.357
-4-4-8:  	12.633
-4-4-16:  	3.927
-4-4-32:  	3.921
-4-4-64:  	3.927
-4-4-128:  	3.927
-4-4-256:  	3.927
-4-4-512:  	3.927
-4-4-1024:  	3.927
-4-4-2048:  	3.927
-8-1-2:  	24.978
-8-2-2:  	5.588
-8-2-4:  	12.633
-8-2-8:  	12.633
-8-2-16:  	24.943
-8-2-32:  	24.943
-8-2-64:  	12.697
-8-2-128:  	24.943
-8-2-256:  	24.943
-8-2-512:  	12.515

# TImulti2

In [7]:
calculate(configurationsMulticlass.configTImulti2)

-2-64:  	86.874
-2-128:  	87.919
-2-256:  	86.616
-2-512:  	21.227
-4-64:  	85.154
-4-128:  	80.094
-4-256:  	24.943
-4-512:  	8.436
-2-128:  	87.919	epoch: 35


# TbaseMulti

In [8]:
calculate(configurationsMulticlass.configTbaseMulti)

-1-128-GRU:  	89.334
-1-128-LSTM:  	88.612
-1-256-GRU:  	89.422
-1-256-LSTM:  	88.424
-1-512-GRU:  	88.888
-1-512-LSTM:  	88.547
-1-1024-GRU:  	88.764
-1-1024-LSTM:  	88.306
-2-128-GRU:  	89.533
-2-128-LSTM:  	89.275
-2-256-GRU:  	89.768
-2-256-LSTM:  	89.246
-2-512-GRU:  	89.257
-2-512-LSTM:  	88.823
-2-1024-GRU:  	88.524
-2-1024-LSTM:  	88.477
-4-128-GRU:  	89.545
-4-128-LSTM:  	89.052
-4-256-GRU:  	89.246
-4-256-LSTM:  	88.559
-4-512-GRU:  	89.504
-4-512-LSTM:  	89.216
-4-1024-GRU:  	88.119
-4-1024-LSTM:  	88.700
-2-256-GRU:  	89.768	epoch: 4


# TMmulti1

In [9]:
calculate(configurationsMulticlass.configTMmulti1)

-2-1-2:  	-1.000
-2-1-4:  	-1.000
-2-1-8:  	-1.000
-2-1-16:  	-1.000
-2-1-32:  	-1.000
-2-1-64:  	-1.000
-2-1-128:  	-1.000
-2-1-256:  	-1.000
-2-1-512:  	-1.000
-2-1-1024:  	-1.000
-2-1-2048:  	-1.000
-2-2-2:  	-1.000
-2-2-4:  	-1.000
-2-2-8:  	-1.000
-2-2-16:  	-1.000
-2-2-32:  	-1.000
-2-2-64:  	-1.000
-2-2-128:  	-1.000
-2-2-256:  	-1.000
-2-2-512:  	-1.000
-2-2-1024:  	-1.000
-2-2-2048:  	-1.000
-2-4-2:  	-1.000
-2-4-4:  	-1.000
-2-4-8:  	-1.000
-2-4-16:  	-1.000
-2-4-32:  	-1.000
-2-4-64:  	-1.000
-2-4-128:  	-1.000
-2-4-256:  	-1.000
-2-4-512:  	-1.000
-2-4-1024:  	-1.000
-2-4-2048:  	-1.000
-4-1-2:  	-1.000
-4-1-4:  	-1.000
-4-1-8:  	-1.000
-4-1-16:  	-1.000
-4-1-32:  	-1.000
-4-1-64:  	-1.000
-4-1-128:  	-1.000
-4-1-256:  	-1.000
-4-1-512:  	-1.000
-4-1-1024:  	-1.000
-4-1-2048:  	-1.000
-4-2-2:  	-1.000
-4-2-4:  	-1.000
-4-2-8:  	-1.000
-4-2-16:  	-1.000
-4-2-32:  	-1.000
-4-2-64:  	-1.000
-4-2-128:  	-1.000
-4-2-256:  	-1.000
-4-2-512:  	-1.000
-4-2-1024:  	-1.000
-4-2-2048:

# TMbaseMulti

In [10]:
calculate(configurationsMulticlass.configTMbaseMulti)

-1-64-GRU:  	82.276
-1-64-LSTM:  	81.458
-1-128-GRU:  	82.620
-1-128-LSTM:  	82.175
-1-256-GRU:  	82.916
-1-256-LSTM:  	81.411
-1-512-GRU:  	79.662
-1-512-LSTM:  	81.464
-1-1024-GRU:  	80.978
-1-1024-LSTM:  	81.037
-2-64-GRU:  	82.780
-2-64-LSTM:  	82.075
-2-128-GRU:  	82.857
-2-128-LSTM:  	81.844
-2-256-GRU:  	80.883
-2-256-LSTM:  	82.745
-2-512-GRU:  	80.895
-2-512-LSTM:  	82.336
-2-1024-GRU:  	81.962
-2-1024-LSTM:  	81.814
-4-64-GRU:  	82.401
-4-64-LSTM:  	80.824
-4-128-GRU:  	81.838
-4-128-LSTM:  	82.626
-4-256-GRU:  	82.537
-4-256-LSTM:  	82.413
-4-512-GRU:  	81.162
-4-512-LSTM:  	82.567
-4-1024-GRU:  	81.648
-4-1024-LSTM:  	24.286
-8-64-GRU:  	-1.000
-8-64-LSTM:  	-1.000
-8-128-GRU:  	-1.000
-8-128-LSTM:  	-1.000
-8-256-GRU:  	-1.000
-8-256-LSTM:  	-1.000
-8-512-GRU:  	-1.000
-8-512-LSTM:  	-1.000
-8-1024-GRU:  	-1.000
-8-1024-LSTM:  	-1.000
-1-256-GRU:  	82.916	epoch: 5


# TMImulti

In [11]:
calculate(configurationsMulticlass.configTMImulti)

-1-64:  	24.019
-1-128:  	23.835
-1-256:  	26.811
-1-512:  	68.002
-2-64:  	24.167
-2-128:  	29.140
-2-256:  	73.165
-2-512:  	64.849
-4-64:  	66.669
-4-128:  	29.218
-4-256:  	24.031
-4-512:  	24.025
-2-256:  	73.165	epoch: 10


# TMTmulti1

In [5]:
calculate(configurationsMulticlass.configTMTmulti1)

-1-64-1-64:  	86.123
-1-64-1-128:  	86.796
-1-64-1-256:  	86.823
-1-64-1-512:  	86.912
-1-64-1-1024:  	86.701
-1-64-2-64:  	85.504
-1-64-2-128:  	86.037
-1-64-2-256:  	86.838
-1-64-2-512:  	86.944
-1-64-2-1024:  	86.384
-1-64-4-64:  	83.924
-1-64-4-128:  	85.213
-1-64-4-256:  	82.078
-1-64-4-512:  	81.980
-1-64-4-1024:  	80.545
-1-128-1-64:  	86.159
-1-128-1-128:  	86.657
-1-128-1-256:  	86.938
-1-128-1-512:  	86.734
-1-128-1-1024:  	86.651
-1-128-2-64:  	85.139
-1-128-2-128:  	85.670
-1-128-2-256:  	86.538
-1-128-2-512:  	86.556
-1-128-2-1024:  	85.196
-1-128-4-64:  	83.417
-1-128-4-128:  	84.873
-1-128-4-256:  	81.852
-1-128-4-512:  	81.906
-1-128-4-1024:  	79.585
-1-256-1-64:  	86.352
-1-256-1-128:  	86.680
-1-256-1-256:  	86.817
-1-256-1-512:  	86.349
-1-256-1-1024:  	85.898
-1-256-2-64:  	85.459
-1-256-2-128:  	86.168
-1-256-2-256:  	86.002
-1-256-2-512:  	86.310
-1-256-2-1024:  	85.172
-1-256-4-64:  	84.345
-1-256-4-128:  	84.561
-1-256-4-256:  	78.213
-1-256-4-512:  	79.321
-1-2

# TSmulti1

In [79]:
calculate(configurationsMulticlass.configTSmulti1)

[64, 0, 256, 128]:   	89.751
[64, 0, 256, 256]:   	89.598
[64, 0, 256, 512]:   	89.657
[64, 0, 256, 1024]:   	88.893
[64, 1, 256, 128]:   	89.604
[64, 1, 256, 256]:   	89.469
[64, 1, 256, 512]:   	89.686
[64, 1, 256, 1024]:   	89.363
[64, 1, 512, 128]:   	89.598
[64, 1, 512, 256]:   	48.019
[64, 1, 512, 512]:   	88.870
[64, 1, 512, 1024]:   	89.322
[64, 1, 1024, 128]:   	89.950
[64, 1, 1024, 256]:   	89.093
[64, 1, 1024, 512]:   	89.369
[64, 1, 1024, 1024]:   	89.580
[128, 0, 256, 128]:   	89.586
[128, 0, 256, 256]:   	89.475
[128, 0, 256, 512]:   	89.598
[128, 0, 256, 1024]:   	89.833
[128, 1, 256, 128]:   	89.674
[128, 1, 256, 256]:   	89.475
[128, 1, 256, 512]:   	89.733
[128, 1, 256, 1024]:   	89.686
[128, 1, 512, 128]:   	89.886
[128, 1, 512, 256]:   	90.079
[128, 1, 512, 512]:   	89.387
[128, 1, 512, 1024]:   	89.099
[128, 1, 1024, 128]:   	48.160
[128, 1, 1024, 256]:   	89.580
[128, 1, 1024, 512]:   	89.222
[128, 1, 1024, 1024]:   	89.340
[256, 0, 256, 128]:   	89.404
[256, 0, 2

# TSMmulti1

In [5]:
calculate(configurationsMulticlass.configTSMmulti1)

-64-0-64-128:  	83.734
-64-0-64-256:  	84.155
-64-0-64-512:  	83.829
-64-0-64-1024:  	82.982
-64-1-64-128:  	83.764
-64-1-64-256:  	83.509
-64-1-64-512:  	83.835
-64-1-64-1024:  	83.159
-64-1-128-128:  	83.539
-64-1-128-256:  	83.657
-64-1-128-512:  	83.480
-64-1-128-1024:  	82.377
-64-1-256-128:  	84.055
-64-1-256-256:  	83.622
-64-1-256-512:  	83.758
-64-1-256-1024:  	83.568
-128-0-64-128:  	83.586
-128-0-64-256:  	82.721
-128-0-64-512:  	83.177
-128-0-64-1024:  	82.662
-128-1-64-128:  	83.414
-128-1-64-256:  	83.456
-128-1-64-512:  	83.557
-128-1-64-1024:  	82.999
-128-1-128-128:  	83.705
-128-1-128-256:  	83.687
-128-1-128-512:  	84.096
-128-1-128-1024:  	83.717
-128-1-256-128:  	83.977
-128-1-256-256:  	83.438
-128-1-256-512:  	83.468
-128-1-256-1024:  	82.175
-256-0-64-128:  	82.519
-256-0-64-256:  	80.699
-256-0-64-512:  	80.972
-256-0-64-1024:  	81.636
-256-1-64-128:  	83.474
-256-1-64-256:  	83.112
-256-1-64-512:  	83.195
-256-1-64-1024:  	83.651
-256-1-128-128:  	84.232
-256-